<a href="https://colab.research.google.com/github/pranavkarnani/StoryGenerator/blob/pranav/BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

In [2]:
! unzip -q /content/plots.zip

unzip:  cannot find or open /content/plots.zip, /content/plots.zip.zip or /content/plots.zip.ZIP.


In [3]:
! pip install rake_nltk

     |████████████████████████████████| 1.5 MB 4.2 MB/s 
     |████████████████████████████████| 749 kB 56.5 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from rake_nltk import Rake
from nltk.tokenize import sent_tokenize
import os
from tqdm import tqdm

infile = '/content/plots'
infile_title = '/content/titles'
outfile = '/content/wikiplot.kwRAKE.tsv'

f = open(infile, 'r', encoding='"ISO-8859-1"')
f_title = open(infile_title, 'r', encoding='"ISO-8859-1"')
fout = open(outfile, 'a', encoding='"ISO-8859-1"')

lines = f.readlines()
lines_title = f_title.readlines()

title_idx = 0
line_idx = 0
my_lines = {}

prev = 0
for line_idx, line in enumerate(tqdm(lines)):
    if "<EOS>" in line:
        article = "".join(lines[prev:line_idx])
        my_lines[title_idx] = article
        prev = line_idx + 1
        title_idx += 1

FileNotFoundError: ignored

In [1]:
import pandas as pd
df = pd.DataFrame()

In [ ]:
df['id'] = my_lines.keys()

In [ ]:
df['text'] = my_lines.values()

In [ ]:
df['title'] = lines_title

In [ ]:
df['title'] = df['title'].apply(lambda x: x[:-1])

In [ ]:
df

In [ ]:
r = Rake()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
for i in tqdm(range(len(lines_title))):
    text = df.loc[i, 'text']
    r.extract_keywords_from_text(text)
    df.loc[i,'outline'] = "##".join(r.get_ranked_phrases()[0:10])

In [ ]:
df

In [ ]:
df.loc[0,'outline']

In [ ]:
df.to_csv('data_20line_outline.csv')

In [ ]:
for i in tqdm(range(len(lines_title))):
    text = df.loc[i, 'text']
    r.extract_keywords_from_text(text)
    df.loc[i,'outline'] = "##".join(r.get_ranked_phrases()[0:20])

In [ ]:
# class myAttention(Attention):
#     def __init__(self, nx, n_ctx, config, scale=False):
#         super(myAttention, self).__init__()

#     def forward(): ### my customized forward pass


# class myBlock(Block):
#     def __init__(self, n_ctx, config, scale=False):
#         super(myBlock, self).__init__()
#     def forward(...):   ### my customized forward pass

# class myGPT2Model(GPT2Model):
#     def __init__(self, config):
#         super(myGPT2Model, self).__init__(config)
#         ....
#         self.apply(self.init_weights)
#     def forward(...).  ### my customized forward pass

In [4]:
data = pd.read_csv('/content/data_10line_outline.csv')
data = data.dropna()
data = data.reset_index()
from tqdm import tqdm

storylines = []

for i in tqdm(range(len(data))):
    outlines = data.loc[i, 'outline'].split("##")
    story = data.loc[i, 'text'].split("\n")
    storyline = [line for line in story for outline in outlines if outline in line]
    storyline = list(set(storyline))
    storylines.append(" <SEP> ".join(storyline))

100%|██████████| 112934/112934 [00:06<00:00, 16870.80it/s]


In [ ]:
! pip install transformers

In [ ]:
! pip install transformers

In [ ]:
data['storyline'] = storylines

In [5]:
data = data.dropna()
data.to_csv('refined.csv')

In [1]:
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
import csv

In [2]:
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [3]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
configuration = BartConfig(vocab_size = 50265, 
                activation_function = 'gelu_new', 
                dropout = 0.1, 
                activation_dropout = 0.1, 
                classifier_dropout = 0.2,
                return_dict=False).from_pretrained('facebook/bart-base', output_hidden_states=True)

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base', config = configuration)
model = model.to(device)

In [4]:
RANDOM_SEED = 73
BATCH_SIZE = 1

EPOCHS = 4
SAMPLE_EVERY = 100

MAX_INPUT_SEQUENCE_LENGTH = 600

In [5]:
from tqdm import tqdm

In [6]:
data = pd.read_csv('/content/refined.csv')

In [53]:
data.columns

Index(['Unnamed: 0', 'index', 'Unnamed: 0.1', 'id', 'text', 'title', 'outline',
       'storyline'],
      dtype='object')

In [7]:
class StoryOutlineDataset(Dataset):

    def __init__(self, data, tokenizer, max_input_length):

        self.data = data
        self.tokenizer = tokenizer

        self.story = []
        self.context = []
        
        for i in tqdm(range(len(self.data))):
            
            context = str(self.data.loc[i, 'title']) + ". " + str(self.data.loc[i, 'storyline'])
            story = self.data.loc[i, 'text']

            encodings_dict_context = tokenizer(context,
                                     truncation=True,
                                     max_length=max_input_length,
                                     padding='max_length'
                                    )

            encodings_dict_story = tokenizer(story,
                                     truncation=True,
                                     max_length=1024,
                                     padding='max_length'
                                    )


            self.context.append(encodings_dict_context)
            self.story.append(encodings_dict_story)
           
            # encoded = tokenizer.encode(context, truncation = True, padding = 'max_length', max_length = max_input_length)
            # mask = tokenizer.get_special_tokens_mask(context, truncation = True, padding = 'max_length', max_length = max_input_length)
            # self.story_id.append(encoded)
            # self.story_mask.append(mask)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, ind):

        item = {}
       
        item['story_id'] = torch.tensor(self.story[ind]['input_ids'])
        item['story_mask'] = torch.tensor(self.story[ind]['attention_mask'])
        
        item['context_id'] = torch.tensor(self.context[ind]['input_ids'])
        item['context_mask'] = torch.tensor(self.context[ind]['attention_mask'])

        return item

In [8]:
story_dataset = StoryOutlineDataset(data, tokenizer, MAX_INPUT_SEQUENCE_LENGTH)

100%|██████████| 112934/112934 [07:45<00:00, 242.77it/s]


In [9]:
from torch.utils.data import random_split

In [10]:
def train_val_split(split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size

In [11]:
train_size, val_size = train_val_split(0.8, story_dataset)
train_dataset, val_dataset = random_split(story_dataset, [train_size, val_size])

In [12]:
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [13]:
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [14]:
learning_rate = 5e-4
eps = 1e-8
warmup_steps = 100

In [15]:
import time
import datetime
scaler = torch.cuda.amp.GradScaler()

In [16]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [17]:
model.cuda()
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)

total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

total_t0 = time.time()
model = model.to(device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [22]:
def format_out_texts(text):
    t_map = tokenizer.special_tokens_map
    for key in t_map:
        text = text.replace(t_map[key], '')
    return text

def inference(encoder_input_id, encoder_attn_mask, tokenizer):
    model.eval()
    story_ids = model.generate(encoder_input_id,
                                    attention_mask = encoder_attn_mask,
                                    num_beams=20,
                                    max_length=400,
                                    temperature=0.9,
                                    top_k=50,
                                    do_sample=True)
    raw_stories = [tokenizer.decode(story) for story in story_ids]
    output_texts = list(map(format_out_texts, raw_stories))
    print(output_texts)
    return output_texts

In [23]:
def train(train_dataloader):

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        
        encoder_input_id = batch['context_id'].to(device)
        encoder_attn_mask = batch['context_mask'].to(device)

        decoder_input_id = batch['story_id'].to(device)
        decoder_attn_mask = batch['story_mask'].to(device)

        if step % SAMPLE_EVERY == 0 and step != 0:
            inference(encoder_input_id, encoder_attn_mask, tokenizer)
            model.train()

        else:
            model.zero_grad()       

            with torch.cuda.amp.autocast():
                outputs = model(input_ids = encoder_input_id,
                                    attention_mask = encoder_attn_mask,
                                    # decoder_input_ids = decoder_input_id[:, :-1],
                                    # decoder_attention_mask = decoder_attn_mask[:, :-1],
                                    labels = decoder_input_id)
                
                loss = outputs.loss
                
            batch_loss = loss.item()
            total_train_loss += batch_loss

            scaler.scale(loss).backward() 
            scaler.step(optimizer) 
            scaler.update()

    avg_train_loss = total_train_loss / len(train_dataloader)


def validate(val_dataloader, file_name):

    print('Evaluating Model')

    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in val_dataloader:
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():        

            outputs  = model(b_input_ids,  
                                attention_mask=b_masks,
                                labels=b_labels)

            loss = outputs[0]  

        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(val_dataloader)
    torch.save(model.state_dict(), '/content/' + file_name)
    return model

In [ ]:
for epoch_i in tqdm(range(0, EPOCHS)):
    print(f'Epoch {epoch_i + 1} of {EPOCHS}')
    train(train_loader)
    # validate(val_loader, 'model.pth')

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1 of 4
['Voices of a Distant Star. As a Special Agent, Mikako pilots a giant robotic mecha called a Tracer as part of a fighting squadron attached to the spacecraft carrier Lysithea. The two continue to communicate across space using the e-mail facilities on their mobile telephones. As the Lysitha travels deeper into space, messages take increasingly longer to reach Noboru on Earth, and the time-lag of their correspondence eventually spans years. As Noboru receives the message almost nine years later.Mikako sends a message to Noboru (which shows the date 2047-09-16), with the subject "I am here", which would reach him eight years later on Earth.']
['Happy Go Lucky (2014 film). Happy Go Lucky is a story of three sisters who desire their life partners to be Inspector, Singer and a NRI. Happy Go Lazy is a Story of Three Sisters Who Want Their Life Partner to Be Inspector, Singer and a RRI. Coincidentally, when they speak of their desires, Bhalla who is the father of three sons stand

 25%|██▌       | 1/4 [3:20:21<10:01:05, 12021.81s/it]

Epoch 2 of 4
["The Speaker in Dreams. In The Speaker in dreams, the town of Brindinford is besieged by evil forces under the command of Ghaerleth Axom. In the Speaker in Dream. The Speaker In Dreams. The speaker in Dreams is interrupted by an attack, which serves as a distraction for the villain's forces to attack the keep of the local baron. This provides an entry point for the player characters into the main quest to discover the secret alliances trying to take over the town. A street fair is interrupted in the middle of the night by a storm of evil forces during the night of the town's battle with the villain’s forces. These are some of the key moments in the battle between the villain and the town:"]
["Bimbo's Initiation. Bimbo is brought through a series of mysterious doors that lead him into yet another sub-basement. Bimbbo flees through various death traps before landing in front of the mysterious order's leader again. Bimsbo still refuses to become a member, but finally accepts